CYPRUS DATASET

Import Packages

In [ ]:
import importlib, parser_inp, fireflow_checker
import pandas as pd

from parser_inp import scan_inp_sections, parse_inp_with_schema, section_to_dataframe
from fireflow_checker import FireFlowChecker
from rtc_prep import (
    load_model, set_pdd_snapshot, attach_hydrant_emitter,
    calibrate_emitter_to_target, link_on_any_source_path,
    sweep_fire_controls, recommend_fire_settings, scenario_metrics,
    cmh_to_gpm, mhead_to_psi
)
from water_benchmark_hub import load
from IPython.display import display

In [86]:
net = load("Network-CY-DBP")
inp_path = net.load()

In [87]:
importlib.reload(parser_inp)

def preview_sections(inp_path,
                     show="both",            # "rowwise", "columnwise", or "both"
                     sample_n=3,             # columnwise: how many rows to show
                     include=None,           # list of section names to include (optional)
                     exclude=None, 
                     exclude_title=True):          # list of section names to exclude (optional)
    include = set(map(str.upper, include or []))
    exclude = set(map(str.upper, exclude or []))

    counts = scan_inp_sections(inp_path)
    schema = parse_inp_with_schema(inp_path, max_samples=0)

    # Sections that are "rowwise" (one logical key/value per line)
    rowwise_names = {"BACKDROP","OPTIONS","REACTIONS","REPORT","TIMES","ENERGY","CONTROLS"}
    rowwise_kinds = {"kv","kv_aligned","kv_aligned_single","controls","backdrop"}

    def classify(sec):
        kind = (schema.get(sec, {}) or {}).get("kind", "unknown")
        if sec in rowwise_names or kind in rowwise_kinds:
            return "rowwise", kind
        return "columnwise", kind

    shown = 0
    for sec in sorted(counts.keys()):

        if exclude_title and sec == "TITLE":
            continue    
        
        if include and sec not in include:   continue
        if sec in exclude:                   continue
        rows = counts[sec]["count"]
        if rows == 0:                        continue

        which, kind = classify(sec)
        if show != "both" and which != show:
            continue

        try:
            df = section_to_dataframe(inp_path, sec)
        except Exception as e:
            print(f"[{sec}] ({which}, kind={kind}) rows={rows} -> parse error: {e}")
            continue

        # Pretty header
        if which == "rowwise":
            print(f"\n[{sec}]  ({which}, kind={kind})  rows={rows}, cols={df.shape[1]}  — showing ALL rows")
            # show whole section without truncation
            with pd.option_context(
                "display.max_rows", None,
                "display.max_columns", None,
                "display.max_colwidth", None,
                "display.width", None
            ):
                display(df)
        else:
            # columnwise: just a small preview
            n = max(1, sample_n)
            print(f"\n[{sec}]  ({which}, kind={kind})  rows={rows}, cols={df.shape[1]}  — showing first {n} rows")
            with pd.option_context(
                "display.max_columns", None,
                "display.max_colwidth", None,
                "display.width", None
            ):
                display(df.head(n))

        shown += 1

    if shown == 0:
        print("No matching sections to show. Adjust filters or thresholds.")
preview_sections(inp_path, show="both", sample_n=3)



[BACKDROP]  (rowwise, kind=kv)  rows=4, cols=9  — showing ALL rows


,Key,Xmin,Ymin,Xmax,Ymax,Units,File,Xoff,Yoff
0,DIMENSIONS,-72.500,-79.500,1522.500,1669.500,,,,
1,UNITS,,,,,None,,,
2,FILE,,,,,,,,
3,OFFSET,,,,,,,0.00,0.00



[CONTROLS]  (rowwise, kind=table_inferred)  rows=2, cols=11  — showing ALL rows


,TargetKind,TargetID,Action,ActionValue,WhenType,CondKind,CondID,Comparator,Threshold,Time,ClockTime
0,LINK,N6,open,,IF,NODE,T_Zone,BELOW,5.5000,,
1,LINK,N6,closed,,IF,NODE,T_Zone,ABOVE,6.5000,,



[COORDINATES]  (columnwise, kind=table)  rows=287, cols=3  — showing first 3 rows


...

,Key,Value1
0,Duration,167:50
1,Hydraulic Timestep,0:05
2,Quality Timestep,0:05
3,Pattern Timestep,0:05
4,Pattern Start,0:00
5,Report Timestep,0:05
6,Report Start,0:00
7,Start ClockTime,0:00:00
8,Statistic,NONE



[VALVES]  (columnwise, kind=table)  rows=4, cols=7  — showing first 3 rows


,ID,Node1,Node2,Diameter,Type,Setting,MinorLoss
0,PRV_dist,dist412,dist412_new,100.0000,PRV,25.0000,0.0000
1,FCV-4,N18,N17,350.0000,FCV,661.0000,0.0000
2,FCV_Desalination,N15_1,N15,800.0000,FCV,0.0000,0.0000



[VERTICES]  (columnwise, kind=table)  rows=26249, cols=3  — showing first 3 rows


,Link,X,Y
0,dist1,1288.180,270.920
1,dist1,1288.180,270.920
2,dist1,1288.180,270.920


In [88]:
importlib.reload(fireflow_checker)

checker = FireFlowChecker(inp_path).run_all()
df = checker.to_dataframe()
display(df)

# optional text report
checker.print_report()


,severity,category,item,detail,suggestion
6,WARN,Connectivity,Multiple disconnected components,"Sizes: [285, 2]",Closed links or missing connections may isolat...
1,WARN,Demand Model,WNTR demand_model,DDA,Switch to PDD for realistic residuals under de...
0,WARN,Units,Flow units,CMH (SI),"If fire-flow targets are in gpm/psi, set repor..."
4,INFO,FCV,FCV_Desalination,Setting<=0 (likely CLOSED).,Ensure connectivity isn’t broken if this is th...
3,INFO,Geometry,Backdrop mismatch,"Network bbox (0.0,0.0)–(1450.0,1723.2) vs back...",Check BACKDROP DIMENSIONS/OFFSET.
5,INFO,Hydrants,No emitter-based hydrants found,Use emitter coefficients or explicit laterals ...,
7,INFO,Status,Closed links at start,"dist1016, dist1022, FCV_Desalination",Confirm intentional closures; these can starve...
2,INFO,Timing,Duration,167:50,"For single-snapshot OFH, consider 'Duration 0:..."


=== Fire-Flow Checker Report v0.2.1 for CY-DBP.inp ===
Totals: 0 ERRORS, 3 WARNINGS, 5 INFOS
severity     category                             item                                                                           detail                                                                                        suggestion
    WARN Connectivity Multiple disconnected components                                                                  Sizes: [285, 2]                                            Closed links or missing connections may isolate areas.
    WARN Demand Model                WNTR demand_model                                                                              DDA                                              Switch to PDD for realistic residuals under deficit.
    WARN        Units                       Flow units                                                                         CMH (SI)                     If fire-flow targets are in gpm/psi, set report c

In [89]:
import wntr, networkx as nx
wn = wntr.network.WaterNetworkModel(str(inp_path))

# Build graph, remove initially CLOSED links (same logic as the checker)
G = wn.get_graph().to_undirected()
for lname in wn.link_name_list:
    l = wn.get_link(lname)
    st = str(getattr(l, "initial_status", getattr(l, "status", ""))).upper()
    if st in {"CLOSED","CLOSE"} and G.has_edge(l.start_node_name, l.end_node_name):
        try: G.remove_edge(l.start_node_name, l.end_node_name)
        except: pass

comps = list(nx.connected_components(G))
comps_sorted = sorted(comps, key=len, reverse=True)
island = comps_sorted[1] if len(comps_sorted) > 1 else set()

print(f"#components = {len(comps_sorted)}")
print(f"Smallest non-main component ({len(island)} nodes):", sorted(list(island))[:30])

# Which CLOSED links sit on the boundary between the main component and others?
closed_links = []
for lname in wn.link_name_list:
    l = wn.get_link(lname)
    st = str(getattr(l, "initial_status", getattr(l, "status", ""))).upper()
    if st in {"CLOSED","CLOSE"}:
        closed_links.append(lname)

cut_links = []
for lname in closed_links:
    l = wn.get_link(lname)
    u, v = l.start_node_name, l.end_node_name
    # if opening this link would join two components, it’s a "cut"
    if (u in comps_sorted[0] and v not in comps_sorted[0]) or (v in comps_sorted[0] and u not in comps_sorted[0]):
        cut_links.append(lname)

print("Closed links that separate components:", cut_links)


#components = 2
Smallest non-main component (2 nodes): ['Desalination', 'N15_1']
Closed links that separate components: ['FCV_Desalination']


C:\Users\mayar\AppData\Local\Temp\ipykernel_66956\2422481109.py:5: DeprecationWarning: wntr.network.WaterNetworkModel.get_graph is deprecated, use wntr.network.WaterNetworkModel.to_graph instead
  G = wn.get_graph().to_undirected()


In [90]:
# 1) Validate
checker = FireFlowChecker(inp_path).run_all()
display(checker.to_dataframe())

# 2) Prepare for RTC (no hydrants yet)
wn = load_model(inp_path)
set_pdd_snapshot(wn, required_m=15.0, pattern_hour=8)

# 3) Add a hydrant proxy when needed
K = attach_hydrant_emitter(wn, "dist1", target_gpm=1000, residual_psi=20)
print("Emitter K:", K)

# 4) Apply modes and get metrics
apply_mode(wn, "fire", {"PRV_set": 25.0, "FCV_Desalination": "Open", "pump_speed": 1.0})
met = scenario_metrics(wn, mode="fire", hydrant_node="dist1", pmax_psi=100.0)
met.maxP_m, met.minP_m, met.Q_hyd_cmh

,severity,category,item,detail,suggestion
6,WARN,Connectivity,Multiple disconnected components,"Sizes: [285, 2]",Closed links or missing connections may isolat...
1,WARN,Demand Model,WNTR demand_model,DDA,Switch to PDD for realistic residuals under de...
0,WARN,Units,Flow units,CMH (SI),"If fire-flow targets are in gpm/psi, set repor..."
4,INFO,FCV,FCV_Desalination,Setting<=0 (likely CLOSED).,Ensure connectivity isn’t broken if this is th...
3,INFO,Geometry,Backdrop mismatch,"Network bbox (0.0,0.0)–(1450.0,1723.2) vs back...",Check BACKDROP DIMENSIONS/OFFSET.
5,INFO,Hydrants,No emitter-based hydrants found,Use emitter coefficients or explicit laterals ...,
7,INFO,Status,Closed links at start,"dist1016, dist1022, FCV_Desalination",Confirm intentional closures; these can starve...
2,INFO,Timing,Duration,167:50,"For single-snapshot OFH, consider 'Duration 0:..."


Emitter K: 60.568969115568926


(116.7904052734375, -5.615234499600774e-07, 492.6265869140625)

In [91]:
import pandas as pd
df = run_two_modes_simple(inp_path, hydrant_node="dist1", prv_candidates_m=[15,20,25,30], pmax_psi=100)
display(df)


,mode,PRV_set_m,maxP_psi,minP_psi,Q_hyd_gpm,maxP_exceed_psi
0,normal,15,166.114964,-7.986739e-07,NaN,66.114964
1,normal,20,166.114964,-7.986739e-07,NaN,66.114964
2,normal,25,166.114964,-7.986739e-07,NaN,66.114964
3,normal,30,166.114964,-7.986739e-07,NaN,66.114964
4,fire,15,166.114964,-7.986739e-07,2168.969589,66.114964
5,fire,20,166.114964,-7.986739e-07,2168.969589,66.114964
6,fire,25,166.114964,-7.986739e-07,2168.969589,66.114964
7,fire,30,166.114964,-7.986739e-07,2168.969589,66.114964
